In [11]:
from jupyter_dash import JupyterDash
from dash import Dash, html, dcc, clientside_callback, ClientsideFunction
from dash.dependencies import Output, Input, State
from datetime import datetime
from dash_extensions import Lottie
import dash_bootstrap_components as dbc
import plotly.express as px
import pandas as pd
from datetime import date
import calendar
from wordcloud import WordCloud
import webbrowser
import numpy as np

# This is for lotties
options = dict(loop=True, autoplay=True, rendererSettings=dict(preserveAspectRatio='xMidYMid slice'))

# JSON URL lotties
LOTTIE_URLS = {
    'connections': "https://lottie.host/8c872553-bf3d-4ef4-9739-01b2718d6c09/q6LtyMNZ28.json",
    'msg_in': "https://assets9.lottiefiles.com/packages/lf20_8wREpI.json",
    'msg_out': "https://assets2.lottiefiles.com/packages/lf20_Cc8Bpg.json",
    'reactions': "https://assets2.lottiefiles.com/packages/lf20_nKwET0.json",
    'companies_stories': "https://lottie.host/7b12b34c-bb3e-4a48-9ca7-71274eaed86d/gnor7nkdhw.json", 
}

# Loading CSV LINKEDIN
df_li_cnt = pd.read_csv("https://raw.githubusercontent.com/busyizzybee/socialMediaEngagement/refs/heads/main/FOLDER/Connections.csv", on_bad_lines='skip')
df_li_invite = pd.read_csv("https://raw.githubusercontent.com/busyizzybee/socialMediaEngagement/refs/heads/main/FOLDER/Invitations.csv", on_bad_lines='skip')
df_li_react = pd.read_csv("https://raw.githubusercontent.com/busyizzybee/socialMediaEngagement/refs/heads/main/FOLDER/Reactions.csv", on_bad_lines='skip')
df_li_msg = pd.read_csv("https://raw.githubusercontent.com/busyizzybee/socialMediaEngagement/refs/heads/main/FOLDER/messages.csv", on_bad_lines='skip')

# Process LinkedIn Dates
# LinkedIn Connections
if "Connected On" in df_li_cnt.columns:
    df_li_cnt["Connected On"] = pd.to_datetime(
        df_li_cnt["Connected On"],
        format="%d-%b-%y",
        errors="coerce"
    )
    df_li_cnt["month"] = df_li_cnt["Connected On"].dt.month.apply(
        lambda x: calendar.month_abbr[int(x)] if pd.notnull(x) and int(x) in range(1, 13) else ""
    )
else:
    df_li_cnt["Connected On"] = pd.NaT

if "Sent At" in df_li_invite.columns:
    df_li_invite["Sent At"] = pd.to_datetime(
        df_li_invite["Sent At"],
        format="%m/%d/%y, %I:%M %p",
        errors="coerce"
    )
else:
    df_li_invite["Sent At"] = pd.NaT

if "Date" in df_li_react.columns:
    df_li_react["Date"] = pd.to_datetime(df_li_react["Date"], errors='coerce')
else:
    df_li_react["Date"] = pd.NaT

if "Date" in df_li_msg.columns:
    df_li_msg["Date"] = pd.to_datetime(df_li_msg["Date"], errors='coerce')
else:
    df_li_msg["Date"] = pd.NaT

#*********************************************************************************************************************************
#                                                               INSTAGRAM
#*********************************************************************************************************************************
# Instagram Data Load
df_ig_cnt = pd.read_csv("https://raw.githubusercontent.com/busyizzybee/socialMediaEngagement/refs/heads/main/Instagram/InstagramConnectionsFollowing.csv", on_bad_lines='skip')
df_ig_cnt_followers = pd.read_csv("https://raw.githubusercontent.com/busyizzybee/socialMediaEngagement/refs/heads/main/Instagram/InstagramConnectionsFollowers.csv", on_bad_lines='skip')
df_ig_invite = pd.read_csv("https://raw.githubusercontent.com/busyizzybee/socialMediaEngagement/refs/heads/main/Instagram/Invitations.csv", on_bad_lines='skip')
df_ig_react = pd.read_csv("https://raw.githubusercontent.com/busyizzybee/socialMediaEngagement/refs/heads/main/Instagram/InstagramReactions.csv", on_bad_lines='skip')
df_ig_msg = pd.read_csv("https://raw.githubusercontent.com/busyizzybee/socialMediaEngagement/refs/heads/main/Instagram/InstagramMessages.csv", on_bad_lines='skip')

# Process Instagram Dates
# Fix 1: Corrected indentation and structural error in date processing logic.
# The original code had a naked 'else' and tried to use a column set to pd.NaT for conversion.
if "Date" in df_ig_cnt_followers.columns:
    df_ig_cnt_followers["Date"] = pd.to_datetime(
        pd.to_numeric(df_ig_cnt_followers["Date"], errors="coerce"),
        unit='s',        # convert from seconds → datetime
        errors="coerce"
    )
    # Assuming 'Connected On' should align with the parsed 'Date' if it exists.
    df_ig_cnt_followers["Connected On"] = df_ig_cnt_followers["Date"]
else:
    # If 'Date' is missing, set both date columns to NaT
    df_ig_cnt_followers["Date"] = pd.NaT
    df_ig_cnt_followers["Connected On"] = pd.NaT

# Fix 2: Applied standard 4-space indentation to the username logic block.

# ensure we have a Username column used by the plotting callback
if "string_list_data__value" in df_ig_cnt_followers.columns:
    df_ig_cnt_followers["Username"] = df_ig_cnt_followers["string_list_data__value"].astype(str)
elif "string_list_data__value".replace("_","") in df_ig_cnt_followers.columns:
    # Fixed the column access here to match the check
    col_name = "string_list_data__value".replace("_","")
    df_ig_cnt_followers["Username"] = df_ig_cnt_followers[col_name].astype(str)
elif "string_list_data__href" in df_ig_cnt_followers.columns:
    # fallback: extract last path segment as a handle if needed
    df_ig_cnt_followers["Username"] = df_ig_cnt_followers["string_list_data__href"].astype(str).str.rstrip("/").str.split("/").str[-1]
elif "string_list_data__value" not in df_ig_cnt_followers.columns and "title" in df_ig_cnt_followers.columns:
    df_ig_cnt_followers["Username"] = df_ig_cnt_followers["title"].astype(str)
else:
    df_ig_cnt_followers["Username"] = df_ig_cnt_followers.iloc[:, 0].astype(str)

# keep a date-only column for comparisons with DatePickerSingle (which returns 'YYYY-MM-DD' strings)
df_ig_cnt_followers["Connected On"] = pd.to_datetime(df_ig_cnt_followers["Connected On"], errors="coerce")
df_ig_cnt_followers["Connected On Date"] = df_ig_cnt_followers["Connected On"].dt.date
# ...existing code...
# keep a date-only column for comparisons with DatePickerSingle (which returns 'YYYY-MM-DD' strings)
df_ig_cnt_followers["Connected On"] = pd.to_datetime(df_ig_cnt_followers["Connected On"], errors="coerce")
df_ig_cnt_followers["Connected On Date"] = df_ig_cnt_followers["Connected On"].dt.date
if "Connected On" in df_ig_cnt.columns:
    df_ig_cnt["Connected On"] = pd.to_datetime(df_ig_cnt["Connected On"], errors='coerce')
    df_ig_cnt["month"] = df_ig_cnt["Connected On"].dt.month.apply(lambda x: calendar.month_abbr[int(x)] if pd.notnull(x) and int(x) in range(1,13) else "")
else:
    df_ig_cnt["Connected On"] = pd.NaT

if "Sent At" in df_ig_invite.columns:
    df_ig_invite["Sent At"] = pd.to_datetime(df_ig_invite["Sent At"], errors='coerce')
else:
    df_ig_invite["Sent At"] = pd.NaT

if "DATE" in df_ig_react.columns:
    df_ig_react["DATE"] = pd.to_datetime(df_ig_react["DATE"])
else:
    df_ig_react["DATE"] = pd.NaT

if "DATE" in df_ig_msg.columns:
    # reload original CSV to ensure we have the raw strings (undo any prior coercion)
    df_ig_msg = pd.read_csv(
        "https://raw.githubusercontent.com/busyizzybee/socialMediaEngagement/refs/heads/main/Instagram/InstagramMessages.csv",
        on_bad_lines='skip'
    )
    # work from the original string column
    raw = df_ig_msg["DATE"].astype(str).str.strip()
    # remove trailing " UTC" then try flexible parse
    raw_stripped = raw.str.replace(" UTC", "", regex=False)
    parsed = pd.to_datetime(raw_stripped, errors='coerce')
    # if still all NaT, try parsing original as UTC (handles " ... UTC")
    if parsed.isna().all():
        parsed = pd.to_datetime(raw, errors='coerce', utc=True)
        try:
            if parsed.dt.tz is not None:
                parsed = parsed.dt.tz_convert(None)
        except Exception:
            pass
    # final fallback: numeric milliseconds since epoch
    if parsed.isna().all():
        parsed = pd.to_datetime(pd.to_numeric(raw, errors='coerce'), unit='ms', errors='coerce')
    df_ig_msg["DATE"] = parsed
else:
    df_ig_msg["DATE"] = pd.NaT


# Global Date Range Calculation
min_date_li = df_li_cnt["Connected On"].min().date() if df_li_cnt["Connected On"].notna().any() else date(2018, 1, 1)
max_date_li = df_li_cnt["Connected On"].max().date() if df_li_cnt["Connected On"].notna().any() else date.today()
min_date_ig = df_ig_cnt["Connected On"].min().date() if df_ig_cnt["Connected On"].notna().any() else date(2018, 1, 1)
max_date_ig = df_ig_cnt["Connected On"].max().date() if df_ig_cnt["Connected On"].notna().any() else date.today()

MIN_DATE = min(min_date_li, min_date_ig)
MAX_DATE = max(max_date_li, max_date_ig)

# APP INITIALIZATION
app = Dash(__name__, external_stylesheets=[dbc.themes.LUX], suppress_callback_exceptions=True)

def create_card(lottie_url, title, id_prefix):
    return dbc.Col([
        dbc.Card([
            dbc.CardHeader(
                html.Div(
                    Lottie(options=options, url=lottie_url),
                    style={"width": "180px", "height": "150px", "margin": "0 auto", "display": "block"}
                )
            ),
            dbc.CardBody([
                html.H6(title),
                html.H2(id=f'{id_prefix}-content', children="000")
            ], style={'textAlign':'center'})
        ], className="h-100")
    ], width=2)

def serve_landing_page():
    """Landing page with LinkedIn and Instagram icons showing project importance"""
    return dbc.Container([
        # Hero Section with Gradient Background
        dbc.Row([
            dbc.Col([
                html.Div([
                    html.Img(src='/assets/download.png', height="180px", className="mb-4", 
                            style={'filter': 'drop-shadow(0 4px 8px rgba(0,0,0,0.2))'}),
                    html.H1("Social Media Analytics Platform", 
                           className="display-3 text-white mb-4",
                           style={'fontWeight': '800', 'textShadow': '2px 2px 4px rgba(0,0,0,0.3)'}),
                    html.P("Transform Your Digital Footprint into Actionable Insights", 
                           className="lead text-white mb-5",
                           style={'fontSize': '1.5rem', 'textShadow': '1px 1px 2px rgba(0,0,0,0.3)'}),
                ], style={'textAlign': 'center', 'padding': '4rem 2rem',
                         'background': 'linear-gradient(135deg, #667eea 0%, #764ba2 100%)',
                         'borderRadius': '15px', 'marginBottom': '3rem'})
            ], width=12)
        ]),
        
        # Platform Selection - MAIN FEATURE
        dbc.Row([
            dbc.Col([
                html.H2("Select Your Platform", className="text-center mb-5", 
                       style={'fontWeight': '700', 'color': '#2c3e50'})
            ], width=12)
        ]),
        
        dbc.Row([
            # LinkedIn Card
            dbc.Col([
                dbc.Card([
                    dbc.CardBody([
                        html.Div([
                            dbc.CardImg(src='assets/linkedinlogo.png', 
                                      style={'height': '160px', 'width': '160px', 
                                            'objectFit': 'contain', 'margin': '0 auto 20px',
                                            'filter': 'drop-shadow(0 4px 6px rgba(0,0,0,0.1))'}),
                        ], className="text-center"),
                        html.H3("LinkedIn Analytics", className="text-center mb-3", 
                               style={'color': '#0077b5', 'fontWeight': '700'}),
                        html.P("Professional Networking Intelligence", 
                              className="text-center text-muted mb-4",
                              style={'fontSize': '1.1rem', 'fontWeight': '500'}),
                        html.Hr(),
                        html.Ul([
                            html.Li("📈 Track connection growth trends", className="mb-2"),
                            html.Li("🏢 Analyze company demographics", className="mb-2"),
                            html.Li("💌 Monitor invitation patterns", className="mb-2"),
                            html.Li("💼 Discover top job titles", className="mb-2"),
                        ], style={'listStyle': 'none', 'padding': '0', 'marginBottom': '1.5rem'}),
                        dbc.Button("Explore LinkedIn Data", 
                                 color="primary", 
                                 href="/linkedin", 
                                 className="w-100", 
                                 size="lg",
                                 style={'fontWeight': '600', 'fontSize': '1.1rem'})
                    ])
                ], className="h-100 shadow-lg", 
                   style={'borderRadius': '15px', 'border': 'none', 'transition': 'transform 0.3s ease',
                         'cursor': 'pointer'})
            ], md=6, className="mb-4"),
            
            # Instagram Card
            dbc.Col([
                dbc.Card([
                    dbc.CardBody([
                        html.Div([
                            dbc.CardImg(src='assets/instagram-logo.png', 
                                      style={'height': '160px', 'width': '160px', 
                                            'objectFit': 'contain', 'margin': '0 auto 20px',
                                            'filter': 'drop-shadow(0 4px 6px rgba(0,0,0,0.1))'}),
                        ], className="text-center"),
                        html.H3("Instagram Analytics", className="text-center mb-3", 
                               style={'color': '#E1306C', 'fontWeight': '700'}),
                        html.P("Social Engagement Metrics", 
                              className="text-center text-muted mb-4",
                              style={'fontSize': '1.1rem', 'fontWeight': '500'}),
                        html.Hr(),
                        html.Ul([
                            html.Li("👥 Monitor follower activity", className="mb-2"),
                            html.Li("💬 Analyze DM trends", className="mb-2"),
                            html.Li("❤️ Track reaction patterns", className="mb-2"),
                            html.Li("🌟 Identify top accounts", className="mb-2"),
                        ], style={'listStyle': 'none', 'padding': '0', 'marginBottom': '1.5rem'}),
                        dbc.Button("Explore Instagram Data", 
                                 color="danger", 
                                 href="/instagram", 
                                 className="w-100", 
                                 size="lg",
                                 style={'fontWeight': '600', 'fontSize': '1.1rem',
                                       'background': 'linear-gradient(45deg, #f09433 0%,#e6683c 25%,#dc2743 50%,#cc2366 75%,#bc1888 100%)',
                                       'border': 'none'})
                    ])
                ], className="h-100 shadow-lg", 
                   style={'borderRadius': '15px', 'border': 'none', 'transition': 'transform 0.3s ease',
                         'cursor': 'pointer'})
            ], md=6, className="mb-4"),
        ], className="mb-5"),
        
        # Why This Matters Section
        dbc.Row([
            dbc.Col([
                html.H2("Why Social Media Analytics Are Essential", 
                       className="text-center mb-5",
                       style={'fontWeight': '800', 'color': '#2c3e50'})
            ], width=12)
        ]),
        
        dbc.Row([
            dbc.Col([
                html.Div([
                    html.Div("📊", style={'fontSize': '4rem', 'marginBottom': '1rem'}),
                    html.H4("Data-Driven Decisions", className="mb-3",
                           style={'fontWeight': '700', 'color': '#667eea'}),
                    html.P("In today's digital landscape, understanding your social media metrics empowers you to make strategic decisions about your online presence, content strategy, and professional networking.",
                          style={'textAlign': 'justify', 'lineHeight': '1.8'})
                ], className="text-center p-4")
            ], md=6, lg=3, className="mb-4"),
            
            dbc.Col([
                html.Div([
                    html.Div("🎯", style={'fontSize': '4rem', 'marginBottom': '1rem'}),
                    html.H4("Personal Branding", className="mb-3",
                           style={'fontWeight': '700', 'color': '#764ba2'}),
                    html.P("Your social media presence is your digital identity. Analytics reveal what resonates with your audience, helping you build a stronger, more authentic personal brand.",
                          style={'textAlign': 'justify', 'lineHeight': '1.8'})
                ], className="text-center p-4")
            ], md=6, lg=3, className="mb-4"),
            
            dbc.Col([
                html.Div([
                    html.Div("🔒", style={'fontSize': '4rem', 'marginBottom': '1rem'}),
                    html.H4("Privacy First", className="mb-3",
                           style={'fontWeight': '700', 'color': '#0077b5'}),
                    html.P("Unlike third-party tools, this platform uses your own data archives. You maintain complete control and ownership while gaining powerful insights.",
                          style={'textAlign': 'justify', 'lineHeight': '1.8'})
                ], className="text-center p-4")
            ], md=6, lg=3, className="mb-4"),
            
            dbc.Col([
                html.Div([
                    html.Div("⚡", style={'fontSize': '4rem', 'marginBottom': '1rem'}),
                    html.H4("Competitive Edge", className="mb-3",
                           style={'fontWeight': '700', 'color': '#E1306C'}),
                    html.P("Understanding engagement patterns and network dynamics gives you an advantage in building meaningful connections and expanding your professional influence.",
                          style={'textAlign': 'justify', 'lineHeight': '1.8'})
                ], className="text-center p-4")
            ], md=6, lg=3, className="mb-4"),
        ], className="mb-5"),
        
        # Statistics Section
        dbc.Row([
            dbc.Col([
                dbc.Card([
                    dbc.CardBody([
                        html.Div([
                            html.H2("2", style={'fontSize': '3.5rem', 'fontWeight': '800', 'color': '#667eea', 'marginBottom': '0'}),
                            html.P("Platforms Supported", className="text-muted", style={'fontSize': '1rem'})
                        ], className="text-center")
                    ])
                ], className="shadow-sm", style={'borderRadius': '10px', 'border': 'none'})
            ], md=3),
            dbc.Col([
                dbc.Card([
                    dbc.CardBody([
                        html.Div([
                            html.H2("10+", style={'fontSize': '3.5rem', 'fontWeight': '800', 'color': '#764ba2', 'marginBottom': '0'}),
                            html.P("Data Metrics", className="text-muted", style={'fontSize': '1rem'})
                        ], className="text-center")
                    ])
                ], className="shadow-sm", style={'borderRadius': '10px', 'border': 'none'})
            ], md=3),
            dbc.Col([
                dbc.Card([
                    dbc.CardBody([
                        html.Div([
                            html.H2("12+", style={'fontSize': '3.5rem', 'fontWeight': '800', 'color': '#0077b5', 'marginBottom': '0'}),
                            html.P("Visualizations", className="text-muted", style={'fontSize': '1rem'})
                        ], className="text-center")
                    ])
                ], className="shadow-sm", style={'borderRadius': '10px', 'border': 'none'})
            ], md=3),
            dbc.Col([
                dbc.Card([
                    dbc.CardBody([
                        html.Div([
                            html.H2("100%", style={'fontSize': '3.5rem', 'fontWeight': '800', 'color': '#E1306C', 'marginBottom': '0'}),
                            html.P("Privacy Focused", className="text-muted", style={'fontSize': '1rem'})
                        ], className="text-center")
                    ])
                ], className="shadow-sm", style={'borderRadius': '10px', 'border': 'none'})
            ], md=3),
        ], className="mb-5"),
        
        # Call to Action
        dbc.Row([
            dbc.Col([
                dbc.Card([
                    dbc.CardBody([
                        html.H3("Ready to Get Started?", 
                               className="text-center text-white mb-4",
                               style={'fontWeight': '700'}),
                        html.P("Select a platform above or learn more about the project and our team.",
                              className="text-center text-white mb-4", 
                              style={'fontSize': '1.2rem'}),
                        html.Div([
                            dbc.Button("About This Project", 
                                     color="light", 
                                     href="/aboutus", 
                                     size="lg", 
                                     className="me-3",
                                     style={'fontWeight': '600'}),
                            dbc.Button("View on GitHub", 
                                     color="dark", 
                                     href="https://github.com/busyizzybee/socialMediaEngagement", 
                                     target="_blank", 
                                     size="lg",
                                     style={'fontWeight': '600'})
                        ], className="text-center")
                    ])
                ], className="shadow-lg", 
                   style={'background': 'linear-gradient(135deg, #667eea 0%, #764ba2 100%)',
                         'borderRadius': '15px', 'border': 'none'})
            ], md=10, className="mx-auto mb-5")
        ]),
        
        # Footer
        dbc.Row([
            dbc.Col([
                html.P([
                    html.Strong("🔐 Your Data, Your Control: "),
                    "This platform analyzes your personal social media archives. Download your data from LinkedIn and Instagram to begin your analytics journey."
                ], className="text-center text-muted", 
                   style={'fontSize': '0.95rem', 'fontStyle': 'italic', 'lineHeight': '1.6'})
            ], md=10, className="mx-auto")
        ])
    ], fluid=True, style={'maxWidth': '1400px', 'margin': '0 auto', 'padding': '2rem'})

def serve_linkedin_layout():
    prefix = 'li'
    return dbc.Container([
        dbc.Row([
            dbc.Col([
                dbc.Card([dbc.CardImg(src='assets/linkedinlogo.png', className='p-3')],className='mb-2'),
                dbc.Card([dbc.CardBody([dbc.CardLink("Project Source", target="_blank", href="https://github.com/busyizzybee/socialMediaEngagement")])]),
            ], width=2),
            dbc.Col([
                dbc.Card([
                    dbc.CardBody([
                        html.H4("LinkedIn Engagement Analysis", className="card-title"),
                        html.P("Filter data by connection date range:", className="card-text"),
                        html.Div([
                            dcc.DatePickerSingle(id=f'{prefix}-date-picker-start', date=MIN_DATE),
                            dcc.DatePickerSingle(id=f'{prefix}-date-picker-end', date=MAX_DATE),
                        ], style={'textAlign': 'center', 'marginTop': '10px'})
                    ])
                ], color="info"),
            ], width=10),
        ],className='mb-2 mt-2'),
        dbc.Row([
            create_card(LOTTIE_URLS['connections'], 'Connections', f'{prefix}-connections'),
            create_card(LOTTIE_URLS['companies_stories'], 'Unique Companies', f'{prefix}-companies'),
            create_card(LOTTIE_URLS['msg_in'], 'Invites Received', f'{prefix}-msg-in'),
            create_card(LOTTIE_URLS['msg_out'], 'Invites Sent', f'{prefix}-msg-out'),
            create_card(LOTTIE_URLS['reactions'], 'Reactions', f'{prefix}-reactions'),
        ], className='mb-2'),
        dbc.Row([
            dbc.Col(dbc.Card(dbc.CardBody([dcc.Graph(id=f'{prefix}-line-chart', figure={}, config={'displayModeBar':False})])), width=6),
            dbc.Col(dbc.Card(dbc.CardBody([dcc.Graph(id=f'{prefix}-bar-chart', figure={}, config={'displayModeBar':False})])), width=6),
        ], className='mb-2'),
        dbc.Row([
            dbc.Col(dbc.Card(dbc.CardBody([dcc.Graph(id=f'{prefix}-pie-chart', figure={})])), width=6),
            dbc.Col(dbc.Card(dbc.CardBody([
                dcc.Graph(id=f'{prefix}-wordcloud', figure={}, config={'displayModeBar': False}),
                html.P("The word cloud shows the most common job titles among connections in the selected period.", 
                       style={'textAlign': 'center', 'fontStyle': 'italic', 'fontSize': '14px'})
            ])), width=6),
        ], className='mb-2'),
    ], fluid=True)

def serve_instagram_layout():
    prefix = 'ig'
    return dbc.Container([
        dbc.Row([
            dbc.Col([
                dbc.Card([dbc.CardImg(src='assets/instagram-logo.png', className='p-3')],className='mb-2'),
                dbc.Card([dbc.CardBody([dbc.CardLink("LinkedIn Dash", target="_blank", href="/linkedin")])]),
            ], width=2),
            dbc.Col([
                dbc.Card([
                    dbc.CardBody([
                        html.H4("Instagram Engagement Analysis", className="card-title"),
                        html.P("Filter data by connection date range:", className="card-text"),
                        html.Div([
                            dcc.DatePickerSingle(id=f'{prefix}-date-picker-start', date=MIN_DATE),
                            dcc.DatePickerSingle(id=f'{prefix}-date-picker-end', date=MAX_DATE),
                        ], style={'textAlign': 'center', 'marginTop': '10px'})
                    ])
                ], color="thistle"),
            ], width=10),
        ],className='mb-2 mt-2'),
        dbc.Row([
            create_card(LOTTIE_URLS['connections'], 'Followers', f'{prefix}-followers'),
            create_card(LOTTIE_URLS['companies_stories'], 'Following', f'{prefix}-following'),
            create_card(LOTTIE_URLS['msg_in'], 'DMs Received', f'{prefix}-msg-in'),
            create_card(LOTTIE_URLS['msg_out'], 'DMs Sent', f'{prefix}-msg-out'),
            create_card(LOTTIE_URLS['reactions'], 'Reactions', f'{prefix}-reactions'),
        ], className='mb-2'),
        dbc.Row([
            dbc.Col(dbc.Card(dbc.CardBody([dcc.Graph(id=f'{prefix}-line-chart', figure={}, config={'displayModeBar':False})])), width=6),
            dbc.Col(dbc.Card(dbc.CardBody([dcc.Graph(id=f'{prefix}-bar-chart', figure={}, config={'displayModeBar':False})])), width=6),
        ], className='mb-2'),
        dbc.Row([
            dbc.Col(dbc.Card(dbc.CardBody([dcc.Graph(id=f'{prefix}-pie-chart', figure={})])), width=6),
            dbc.Col(dbc.Card(dbc.CardBody([
               dcc.Graph(id=f'{prefix}-recent-followers-bar', figure={}, config={'displayModeBar': False}),
                html.P("Top 10 Most Recent Followers", 
                    style={'textAlign': 'center', 'fontStyle': 'italic', 'fontSize': '14px'})
            ])), width=6),
        ], className='mb-2'),
    ], fluid=True)

def serve_aboutus_layout():
    GITHUB_URL = "https://github.com/busyizzybee/socialMediaEngagement"
    profile_img_style = {
        "width": "120px", "height": "120px", "borderRadius": "50%", 
        "objectFit": "cover", "marginBottom": "10px", "border": "2px solid #007bff"
    }
    justify_style = {'textAlign': 'justify'}

    return dbc.Container([
        dbc.Row([
            dbc.Col(width=1),
            dbc.Col([html.Img(src='/assets/download.png', height="180px", id="connectly-logo", style={"display": "block"})], 
                    width="auto", className="d-flex align-items-center"),
            dbc.Col([html.H2("SOCIAL MEDIA ANALYSIS MANAGEMENT PLATFORM", className="text-center text-primary")], 
                    width=7, className="d-flex align-items-center ml-auto"),
            dbc.Col(width=2)
        ], className="mb-5 mt-5 d-flex align-items-center border-bottom pb-4"),
        
        dbc.Row([
            dbc.Col([
                dbc.Card(dbc.CardBody([
                    html.H4("PROJECT OVERVIEW", className="card-title"),
                    html.P([
                        "This Social Media Analytics Dashboard was developed to visualize and analyze personal social media engagement data from ",
                        html.Strong("LinkedIn"),
                        " and ",
                        html.Strong("Instagram"),
                        ". The goal is to provide a comprehensive, data-driven view of connection trends, messaging activity, and content reactions. By leveraging data from these platforms, users can gain insights to improve their digital presence and professional networking strategies."
                    ], className="card-text", style=justify_style),
                    html.P(["Check out the full source code and documentation on GitHub: ", 
                            html.A("Project Repository", href=GITHUB_URL, target="_blank", className="text-info")], 
                           className="card-text mt-3")
                ]))
            ], width=9, className="mx-auto mb-4"),
        ]),
        
        dbc.Row([dbc.Col([html.H3("TEAM & DEVELOPMENT", className="text-center mb-3 mt-3")], width=12, className="mx-auto mt-5")]),
        
        dbc.Row([
            dbc.Col(dbc.Card(dbc.CardBody([
                html.Div(html.Img(src='/assets/Blessy.png', style=profile_img_style), className="d-flex justify-content-center"),
                html.H5("BLESSY SOLAS", className="card-title text-success"),
                html.P("Lead Developer", className="card-text"),
                html.P("Primary Data Scientist & Dash Implementer", className="card-text-small text-muted"),
            ]), className="text-center h-100"), width=4, className="mb-4"),
            dbc.Col(dbc.Card(dbc.CardBody([
                html.Div(html.Img(src='/assets/Neo.png', style=profile_img_style), className="d-flex justify-content-center"),
                html.H5("NEO RAY SERRANO", className="card-title text-success"),
                html.P("Team Member", className="card-text"),
                html.P("Data Acquisition & Processing", className="card-text-small text-muted"),
            ]), className="text-center h-100"), width=4, className="mb-4"),
            dbc.Col(dbc.Card(dbc.CardBody([
                html.Div(html.Img(src='/assets/Gerald.png', style=profile_img_style), className="d-flex justify-content-center"),
                html.H5("GERALD ELLI RAMOS", className="card-title text-success"),
                html.P("Team Member", className="card-text"),
                html.P("Data Analysis & Reporting", className="card-text-small text-muted"),
            ]), className="text-center h-100"), width=4, className="mb-4"),
        ], className="d-flex justify-content-center", style={'width': '100%'}),
        
        dbc.Row([
            dbc.Col(dbc.Card(dbc.CardBody([
                html.Div(html.Img(src='/assets/Justin.png', style=profile_img_style), className="d-flex justify-content-center"),
                html.H5("JUSTIN AARON SALONGA", className="card-title text-success"),
                html.P("Team Member", className="card-text"),
                html.P("Visualization & Dashboard Design", className="card-text-small text-muted"),
            ]), className="text-center h-100"), width=4, className="mb-4"),
            dbc.Col(dbc.Card(dbc.CardBody([
                html.Div(html.Img(src='/assets/Sean.png', style=profile_img_style), className="d-flex justify-content-center"),
                html.H5("SEAN CALVIN SOLIVEN", className="card-title text-success"),
                html.P("Team Member", className="card-text"),
                html.P("Dash Implementation & Deployment", className="card-text-small text-muted"),
            ]), className="text-center h-100"), width=4, className="mb-4"),
        ], className="d-flex justify-content-center mx-auto", style={'width': '85%'}),
        
        dbc.Row([
            dbc.Col([
                html.H3("FRAMEWORKS", className="text-center mb-3 mt-4"),
                dbc.Card(dbc.CardBody([
                    html.H5("Core Technologies", className="card-title text-success"),
                    html.P("Python, Dash, Plotly, Pandas, Dash Bootstrap Components", className="card-text"),
                    html.P("Built for speed and clean visualization.", className="card-text-small text-muted"),
                ]), className="text-center")
            ], width=6, className="mx-auto mb-4"),
        ]),
        
        dbc.Row([
            dbc.Col(html.P("Note: This dashboard is a personal analytics tool. Data displayed is based on the user's downloadable archive data.", 
                          style={'textAlign': 'center', 'fontStyle': 'italic', 'fontSize': '12px'}, className="mt-4"), width=12)
        ])
    ], fluid=True)

# Main App Layout with Night Mode
app.layout = html.Div([
    dcc.Location(id='url', refresh=False),
    dcc.Store(id='night-mode-store', storage_type='local', data=False),
    
    dbc.NavbarSimple(
        children=[
            dbc.NavItem(dbc.NavLink("Home", href="/", style={'fontSize': '1.15rem', 'marginRight': '1rem'})),
            dbc.NavItem(dbc.NavLink("LinkedIn", href="/linkedin", style={'fontSize': '1.15rem', 'marginRight': '1rem'})),
            dbc.NavItem(dbc.NavLink("Instagram", href="/instagram", style={'fontSize': '1.15rem', 'marginRight': '1rem'})),
            dbc.NavItem(dbc.NavLink("About Us", href="/aboutus", style={'fontSize': '1.15rem', 'marginRight': '1rem'})),
            
        ],
        brand=html.Span("Social Media Analytics", style={'fontSize': '1.75rem', 'fontWeight': '700', 'paddingLeft': '1rem'}),
        brand_href="/",
        color="primary",
        dark=True,
        className="mb-4",
    ),
    
    # Night Mode Toggle Button
    html.Button("🌙", id="night-mode-toggle", n_clicks=0),
    
    html.Div(id='page-content')
])

# Clientside callback for night mode
app.clientside_callback(
    """
    function(n_clicks, stored) {
        if (n_clicks === 0) {
            if (stored) {
                document.body.classList.add('night-mode');
                return [true, '☀️'];
            }
            return [false, '🌙'];
        }
        const isNightMode = !stored;
        if (isNightMode) {
            document.body.classList.add('night-mode');
            return [true, '☀️'];
        } else {
            document.body.classList.remove('night-mode');
            return [false, '🌙'];
        }
    }
    """,
    [Output('night-mode-store', 'data'), Output('night-mode-toggle', 'children')],
    [Input('night-mode-toggle', 'n_clicks')],
    [State('night-mode-store', 'data')]
)

# Routing Callback
@app.callback(Output('page-content', 'children'), [Input('url', 'pathname')])
def display_page(pathname):
    if pathname == '/' or pathname == '/home':
        return serve_landing_page()
    elif pathname == '/linkedin':
        return serve_linkedin_layout()
    elif pathname == '/instagram':
        return serve_instagram_layout()
    elif pathname == '/aboutus':
        return serve_aboutus_layout()
    else:
        return html.Div(dbc.Alert(f"404 - Page not found: {pathname}", color="danger"))

# LinkedIn Callbacks
@app.callback(
    Output('li-connections-content','children'), Output('li-companies-content','children'),
    Output('li-msg-in-content','children'), Output('li-msg-out-content','children'),
    Output('li-reactions-content','children'),
    [Input('li-date-picker-start','date'), Input('li-date-picker-end','date')],
)
def update_li_small_cards(start_date, end_date):
    if not (start_date and end_date):
        return 0, 0, 0, 0, 0
    dff_c = df_li_cnt[(df_li_cnt['Connected On'] >= start_date) & (df_li_cnt['Connected On'] <= end_date)]
    conctns_num = len(dff_c)
    compns_num = len(dff_c['Company'].dropna().unique())
    dff_i = df_li_invite[(df_li_invite['Sent At'] >= start_date) & (df_li_invite['Sent At'] <= end_date)]
    in_num = len(dff_i[dff_i['Direction'] == 'INCOMING'])
    out_num = len(dff_i[dff_i['Direction'] == 'OUTGOING'])
    dff_r = df_li_react[(df_li_react['Date'] >= start_date) & (df_li_react['Date'] <= end_date)]
    reactns_num = len(dff_r)
    return conctns_num, compns_num, in_num, out_num, reactns_num

@app.callback(Output('li-line-chart','figure'), [Input('li-date-picker-start','date'), Input('li-date-picker-end','date')])
def update_li_line(start_date, end_date):
    if not (start_date and end_date):
        return {}
    dff = df_li_cnt[(df_li_cnt['Connected On'].astype(str) >= start_date) & (df_li_cnt['Connected On'].astype(str) <= end_date)]
    month_counts = dff['month'].value_counts()
    month_order = list(calendar.month_abbr)[1:]
    plot_df = pd.DataFrame({'month': month_order}).merge(month_counts.rename('Total connections').reset_index(), on='month', how='left').fillna(0)
    plot_df['month'] = pd.Categorical(plot_df['month'], categories=month_order, ordered=True)
    plot_df = plot_df.sort_values('month')
    fig_line = px.line(plot_df, x='month', y='Total connections', template='ggplot2', title="Total Connections by Month")
    fig_line.update_traces(mode="lines+markers", fill='tozeroy', line={'color':'blue'})
    fig_line.update_layout(margin=dict(l=20, r=20, t=30, b=20), xaxis_title=None)
    return fig_line

@app.callback(Output('li-bar-chart','figure'), [Input('li-date-picker-start','date'), Input('li-date-picker-end','date')])
def update_li_bar(start_date, end_date):
    if not (start_date and end_date):
        return {}
    dff = df_li_cnt[(df_li_cnt['Connected On'].astype(str) >= start_date) & (df_li_cnt['Connected On'].astype(str) <= end_date)]
    dff = dff.dropna(subset=['Company'])
    company_counts = dff['Company'].value_counts().head(6).reset_index()
    company_counts.columns = ['Company', 'Total connections']
    fig_bar = px.bar(company_counts, x='Total connections', y='Company', template='ggplot2', orientation='h', title="Top 6 Companies by Connections")
    fig_bar.update_yaxes(tickangle=0)
    fig_bar.update_layout(margin=dict(l=20, r=20, t=30, b=20), yaxis_title=None)
    fig_bar.update_traces(marker_color='darkblue')
    return fig_bar

@app.callback(Output('li-pie-chart','figure'), [Input('li-date-picker-start','date'), Input('li-date-picker-end','date')])
def update_li_pie(start_date, end_date):
    if not (start_date and end_date):
        return {}
    dff = df_li_msg[(df_li_msg['DATE'].astype(str) >= start_date) & (df_li_msg['DATE'].astype(str) <= end_date)]
    msg_sent = len(dff[dff['FROM'] == 'Adam Schroeder'])
    msg_rcvd = len(dff[dff['FROM'] != 'Adam Schroeder'])
    if msg_sent + msg_rcvd == 0:
        return {}
    fig_pie = px.pie(names=['Sent','Received'], values=[msg_sent, msg_rcvd], template='ggplot2', title="Messages Sent & Received")
    fig_pie.update_layout(margin=dict(l=20, r=20, t=30, b=20))
    fig_pie.update_traces(marker_colors=['#0077b5','#00a0dc'])
    return fig_pie

@app.callback(Output('li-wordcloud','figure'), [Input('li-date-picker-start','date'), Input('li-date-picker-end','date')])
def update_li_wordcloud(start_date, end_date):
    if not (start_date and end_date):
        return {}
    dff = df_li_cnt[(df_li_cnt['Connected On'].astype(str) >= start_date) & (df_li_cnt['Connected On'].astype(str) <= end_date)]
    dff_positions = dff['Position'].dropna().astype(str)
    if dff_positions.empty:
        return px.imshow(np.zeros((10,10)), color_continuous_scale='gray', title="No Position Data Found").update_xaxes(visible=False).update_yaxes(visible=False).update_layout(margin=dict(l=20, r=20, t=40, b=20))
    my_wordcloud = WordCloud(background_color='white', width=800, height=400, colormap='winter').generate(' '.join(dff_positions))
    img_array = my_wordcloud.to_array()
    fig_wordcloud = px.imshow(img_array, title="Most Frequent Job Titles")
    fig_wordcloud.update_layout(margin=dict(l=20, r=20, t=40, b=20), xaxis_visible=False, yaxis_visible=False)
    return fig_wordcloud

# Instagram Callbacks
@app.callback(
    Output('ig-followers-content','children'), Output('ig-following-content','children'),
    Output('ig-msg-in-content','children'), Output('ig-msg-out-content','children'),
    Output('ig-reactions-content','children'),
    [Input('ig-date-picker-start','date'), Input('ig-date-picker-end','date')],
)
def update_ig_small_cards(start_date, end_date):
    if not (start_date and end_date):
        return 0, 0, 0, 0, 0
    dff_c = df_ig_cnt[(df_ig_cnt['Connected On'].astype(str) >= start_date) & (df_ig_cnt['Connected On'].astype(str) <= end_date)]
    following_num = len(dff_c)
    followers_num = len(df_ig_cnt['value'].unique())
    dff_i = df_ig_invite[(df_ig_invite['Sent At'].astype(str) >= start_date) & (df_ig_invite['Sent At'].astype(str) <= end_date)]
    in_num = len(dff_i[dff_i['Direction'] == 'INCOMING'])
    out_num = len(dff_i[dff_i['Direction'] == 'OUTGOING'])
    dff_r = df_ig_react[(df_ig_react['DATE'].astype(str) >= start_date) & (df_ig_react['DATE'].astype(str) <= end_date)]
    reactns_num = len(dff_r)
    return followers_num, following_num, in_num, out_num, reactns_num

#********************************************************************************************
#                                       line chart
# how many ig you have followed
#********************************************************************************************
@app.callback(Output('ig-line-chart','figure'), [Input('ig-date-picker-start','date'), Input('ig-date-picker-end','date')])
def update_ig_line(start_date, end_date):
    if not (start_date and end_date):
        return {}
    dff = df_ig_cnt[(df_ig_cnt['Connected On'].astype(str) >= start_date) & (df_ig_cnt['Connected On'].astype(str) <= end_date)]
    month_counts = dff['month'].value_counts()
    month_order = list(calendar.month_abbr)[1:]
    plot_df = pd.DataFrame({'month': month_order}).merge(month_counts.rename('Total following').reset_index(), on='month', how='left').fillna(0)
    plot_df['month'] = pd.Categorical(plot_df['month'], categories=month_order, ordered=True)
    plot_df = plot_df.sort_values('month')
    fig_line = px.line(plot_df, x='month', y='Total following', template='ggplot2', title="Following Activity by Month")
    fig_line.update_traces(mode="lines+markers", fill='tozeroy', line={'color':'#E1306C'})
    fig_line.update_layout(margin=dict(l=20, r=20, t=30, b=20), xaxis_title=None)
    return fig_line

#********************************************************************************************
#                                       bar chart
# 
#********************************************************************************************
@app.callback(Output('ig-bar-chart','figure'), 
              [Input('ig-date-picker-start','date'), 
               Input('ig-date-picker-end','date')])
def update_ig_bar(start_date, end_date):
    # We need pandas imported inside the function for safety and the new logic
    import pandas as pd 
    import plotly.express as px
    import datetime

    # 1. Input Validation
    if not (start_date and end_date):
        # Return an empty figure if dates aren't selected
        return {}
    
    # 2. Convert date strings to datetime objects for accurate comparison
    start_dt = datetime.datetime.strptime(start_date, '%Y-%m-%d').date()
    end_dt = datetime.datetime.strptime(end_date, '%Y-%m-%d').date()

    # 3. Filter DataFrame based on the 'Connected On' date column
    # Ensure the 'Connected On' column is stored as date objects (or at least comparable strings/dates)
    # Note: If 'Connected On' is already a datetime object, you should compare the .date() component
    dff = df_ig_cnt[
        (pd.to_datetime(df_ig_cnt['Connected On']).dt.date >= start_dt) & 
        (pd.to_datetime(df_ig_cnt['Connected On']).dt.date <= end_dt)
    ].copy()

    # 4. Clean Data
    # The 'value' column holds the user name/handle. Dropping NaNs ensures valid entries are counted.
    dff = dff.dropna(subset=['value', 'timestamp'])

    # 5. Find the Top 5 Most Recent Follows
    if dff.empty:
        # Return an empty figure if no data is found in the range
        return {}
    
    # Convert timestamp (Unix epoch) to a datetime object for accurate comparison and sorting
    dff['dt_timestamp'] = pd.to_datetime(dff['timestamp'], unit='s')
    
    # Sort by timestamp descending and take the top 5
    top_5_follows = dff.sort_values('dt_timestamp', ascending=False).head(5)
    
    # Prepare the data frame for visualization
    recent_follow_df = pd.DataFrame({
        # Prepend '@' to the user handle for display
        'User': top_5_follows['value'].apply(lambda x: f"@{x}"),
        # Use a nominal value of 1 for the bar length, simply to visualize the list
        'Recency Index': [1] * len(top_5_follows), 
        # Add the full timestamp string for display in the hover text, formatted nicely
        'Time': top_5_follows['dt_timestamp'].dt.strftime('%Y-%m-%d %H:%M:%S')
    })
    
    # Reverse the DataFrame so the most recent is at the top of the chart
    recent_follow_df = recent_follow_df.iloc[::-1]

    # 6. Create Plotly Bar Chart (for the top 5 most recent entries)
    fig_bar = px.bar(
        recent_follow_df, 
        x='Recency Index', 
        y='User', 
        template='plotly_white', 
        orientation='h', 
        # Use a title that clearly communicates the result
        title="Top 5 Most Recent Follows",
        # Only show the full 'Time' when hovering
        hover_data={'Recency Index': False, 'User': False, 'Time': True}
    )

    # 7. Update Layout and Aesthetics
    fig_bar.update_yaxes(tickangle=0)
    fig_bar.update_layout(
        margin=dict(l=20, r=20, t=50, b=20), 
        # Hide the x-axis for this single-value visualization
        xaxis_title=None,
        xaxis=dict(showticklabels=False, zeroline=False, showgrid=False), 
        yaxis_title=None,
        # Customize title appearance
        title_font_size=16,
        title_x=0.5 # Center the title
    )
    fig_bar.update_traces(marker_color="#F97051", marker_line_color='black', marker_line_width=0.5)
    
    return fig_bar
#********************************************************************************************
#                                       pie chart    
# track who sent message 
#********************************************************************************************

@app.callback(Output('ig-pie-chart','figure'),
              [Input('ig-date-picker-start','date'), Input('ig-date-picker-end','date')])
def update_ig_pie(start_date, end_date):
    if not (start_date and end_date):
        return {}
    # filter by parsed datetimes; guard against NaT column
    try:
        start_dt = pd.to_datetime(start_date)
        end_dt = pd.to_datetime(end_date)
    except:
        return {}
    dff = df_ig_msg[(df_ig_msg['DATE'] >= start_dt) & (df_ig_msg['DATE'] <= end_dt)]
    if dff.empty:
        # return a blank/placeholder figure so the dashboard doesn't error
        return px.imshow(np.zeros((10,10)), color_continuous_scale='gray',
                         title="No DM data for selected date range").update_xaxes(visible=False).update_yaxes(visible=False).update_layout(margin=dict(l=20,r=20,t=40,b=20))
    # detect sender column (support different naming)
    possible_sender_cols = [c for c in df_ig_msg.columns if any(k in c.lower() for k in ['sender','from','messages__sender','messages_sender'])]
    if not possible_sender_cols:
        return px.imshow(np.zeros((10,10)), color_continuous_scale='gray',
                         title="No sender column found in Instagram messages").update_xaxes(visible=False).update_yaxes(visible=False).update_layout(margin=dict(l=20,r=20,t=40,b=20))
    sender_col = possible_sender_cols[0]
    user_name = 'Sean Calvin Soliven'
    msg_sent = len(dff[dff[sender_col] == user_name])
    msg_rcvd = len(dff[dff[sender_col] != user_name])
    if msg_sent + msg_rcvd == 0:
        return px.imshow(np.zeros((10,10)), color_continuous_scale='gray',
                         title="No messages from/to specified user in range").update_xaxes(visible=False).update_yaxes(visible=False).update_layout(margin=dict(l=20,r=20,t=40,b=20))
    fig_pie = px.pie(names=['Sent','Received'], values=[msg_sent, msg_rcvd], template='ggplot2', title="Direct Messages Sent & Received")
    fig_pie.update_layout(margin=dict(l=20, r=20, t=30, b=20))
    fig_pie.update_traces(marker_colors=['#F56040',"#833AB4"])
    return fig_pie

#********************************************************************************************
#                                       bar chart
#********************************************************************************************
@app.callback(
    Output('ig-recent-followers-bar', 'figure'),
    [Input('ig-date-picker-start', 'date'),
     Input('ig-date-picker-end', 'date')]
)
def update_recent_followers_bar(start_date, end_date):
    if not (start_date and end_date):
        return {}
    
    dff = df_ig_cnt_followers[
        (df_ig_cnt_followers['Connected On'].astype(str) >= start_date) &
        (df_ig_cnt_followers['Connected On'].astype(str) <= end_date)
    ]
    # convert picker strings to date objects
    try:
        start_dt = pd.to_datetime(start_date).date()
        end_dt = pd.to_datetime(end_date).date()
    except Exception:
        return {}

    # filter using the prepared date-only column
    dff = df_ig_cnt_followers[
        (df_ig_cnt_followers['Connected On Date'] >= start_dt) &
        (df_ig_cnt_followers['Connected On Date'] <= end_dt)
    ].copy()
     
     # Get 10 most recent followers
    top10 = dff.sort_values(by="Connected On", ascending=False).head(10)
 
    fig = px.bar(
         top10,
        x="Username",  # adjust if your CSV column is different, example: 'Profile Name'
        y="Connected On",
         title="Top 10 Most Recent Followers",
         text="Connected On"
     )
 
    fig.update_layout(xaxis={'categoryorder':'total descending'})
    fig.update_layout(margin=dict(l=20, r=20, t=30, b=20))
    fig.update_traces(marker_color="#833AB4")
    return fig


# RUN THE APP
app.run(mode='jupyterlab', port=8009)
webbrowser.open("http://127.0.0.1:8009")

True